In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = f'drive/MyDrive/datasets/Копия отправка_комбинации поддержка после ПХТ 3 стадия.xlsx'

In [ ]:
data = pd.read_excel(file_path)

In [ ]:
data.head(2)

,id_ответа,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,...,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий
0,1,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,до 42 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0
1,2,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,от 43 до 60 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0


In [ ]:
data.drop(columns=['id_ответа', 'Комментарий', 'Альтернатива, если 50% - обязательно'], inplace=True)

In [ ]:
for col in data.columns:
    print(f'{col}: {data[col].unique()}')

Уникальные значения столбца Раса: ['Азиатская' 'Европейская' 'Другая']
Уникальные значения столбца Пол: [' Мужской' ' Женский']
Уникальные значения столбца Возраст: [' До 70' ' >70']
Уникальные значения столбца Статус курения: [' В настоящее время'
 ' Курение в прошлом (бросил более 1 месяца до 1 года)'
 ' Курение в прошлом (бросил более 1 года)']
Уникальные значения столбца ECOG: [' 0-1' 2]
Уникальные значения столбца Есть опухолевая нагрузка? (симптомная опухоль): [' да' ' нет']
Уникальные значения столбца Ко-мутации KRAS: [' да' ' нет']
Уникальные значения столбца Ко-мутации p53.: [' да' ' нет']
Уникальные значения столбца Ко-мутации STK11: [' да' ' нет']
Уникальные значения столбца Ко-мутации KEAP1: [' да' ' нет']
Уникальные значения столбца Срок от окончания ХЛТ: ['до 42 дней' 'от 43 до 60 дней' 'более 61 дня']
Уникальные значения столбца Молекулярный статус (только для неплоскоклеточного рака): [' нет мутаций' ' не исследовались' ' EGFR редкий вариант' ' EGFR ex19'
 ' EGFR ex21' 

In [ ]:
df = data.copy()

In [ ]:
df['ECOG'] = df['ECOG'].astype(str)

In [ ]:
df.ECOG.unique()

array([' 0-1', '2'], dtype=object)

In [ ]:
df.dtypes

Раса                                                        object
Пол                                                         object
Возраст                                                     object
Статус курения                                              object
ECOG                                                        object
Есть опухолевая нагрузка? (симптомная опухоль)              object
Ко-мутации KRAS                                             object
Ко-мутации p53.                                             object
Ко-мутации STK11                                            object
Ко-мутации KEAP1                                            object
Срок от окончания ХЛТ                                       object
Молекулярный статус (только для неплоскоклеточного рака)    object
PD-L1 статус                                                object
Предпочтение пациента по ответу на терапию                  object
Ответ эксперта (Лактионов)                                  ob

In [ ]:
# Разделяем на фичи и таргеты (отдельно ответ эксперта, отдельно уверенность)

X = df.drop(['Ответ эксперта (Лактионов)', 'Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%'], axis=1)
y_treatment = df['Ответ эксперта (Лактионов)']
y_confidence = df[['Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%']]

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_treatment_train, y_treatment_test, y_confidence_train, y_confidence_test = train_test_split(
    X, y_treatment, y_confidence, test_size=0.2, random_state=42
)

# Создаем трансформаторы для числовых и категориальных признаков
categorical_features = X.select_dtypes(include=['object']).columns

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Комбинируем трансформаторы
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Создаем модели
treatment_model = RandomForestClassifier(n_estimators=100, random_state=42)
confidence_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Создаем пайплайн для каждой модели
treatment_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', treatment_model)
])

confidence_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', confidence_model)
])

# Обучаем модели
treatment_pipeline.fit(X_train, y_treatment_train)
confidence_pipeline.fit(X_train, y_confidence_train)

# Делаем предсказания
y_treatment_pred = treatment_pipeline.predict(X_test)
y_confidence_pred = confidence_pipeline.predict(X_test)

# Оцениваем модели
print("Accuracy for Treatment Recommendation:", accuracy_score(y_treatment_test, y_treatment_pred))
print("Classification Report for Treatment Recommendation:\n", classification_report(y_treatment_test, y_treatment_pred))

for i, col in enumerate(y_confidence.columns):
    print(f"\nAccuracy for Confidence Level '{col}':", accuracy_score(y_confidence_test[col], y_confidence_pred[:, i]))
    print(f"Classification Report for Confidence Level '{col}':\n",
          classification_report(y_confidence_test[col], y_confidence_pred[:, i]))


Accuracy for Treatment Recommendation: 1.0
Classification Report for Treatment Recommendation:
               precision    recall  f1-score   support

   Алектиниб       1.00      1.00      1.00      4562
  Дурвалумаб       1.00      1.00      1.00      8757
  Наблюдение       1.00      1.00      1.00     28085
 Осимертиниб       1.00      1.00      1.00      8363

    accuracy                           1.00     49767
   macro avg       1.00      1.00      1.00     49767
weighted avg       1.00      1.00      1.00     49767


Accuracy for Confidence Level 'Поставьте галочку, если уверены на 100%': 1.0
Classification Report for Confidence Level 'Поставьте галочку, если уверены на 100%':
               precision    recall  f1-score   support

       False       1.00      1.00      1.00     31853
        True       1.00      1.00      1.00     17914

    accuracy                           1.00     49767
   macro avg       1.00      1.00      1.00     49767
weighted avg       1.00      1.0